In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import requests
import json
import time

SCOPES = 'https://www.googleapis.com/auth/spreadsheets'

#global browser
#chrome_options = webdriver.ChromeOptions() #新增chrome選項
#chrome_options.add_argument('--headless') #隱藏GUI
#chrome_options.add_argument('--disable-gpu')
#browser = webdriver.Chrome('chromedriver.exe',chrome_options = chrome_options) 


resultdata = getprdstock("4562351019316")
#resultdata = getsheet('商品ID!A:Q') #獲取試算表所有資料        




#browser.quit()


蝦皮錯誤
{'spreadsheetId': '19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI', 'updatedRange': "'庫存表'!G1124:K1124", 'updatedRows': 1, 'updatedColumns': 5, 'updatedCells': 5}


NameError: name 'browser' is not defined

In [2]:
def getprdstock(isoinput):
    prdidData = getsheet('商品ID!A:Q') #獲取試算表所有資料
    #提取ISO
    isoAry = []
    wval=["","","","","",""]
    for row in range(len(prdidData)):
        try:
            isoAry.append(prdidData[row][0])#iso位置
        except:
            isoAry.append("")#如果錯誤(空值)，新增空白
    getArow = isoAry.index(isoinput) #找到list的第幾個
    IDObj = {"Yahoo":{"web":"","type":""},"Ruten":{"web":"","type":""},"Shopee":{"web":"","type":""},"PCt":{"web":"","type":""},"PCd":{"web":"","type":""},"pcone":{"web":"","type":""}}
    try:#webid
        IDObj["Yahoo"]["web"]=prdidData[getArow][4]
    except:#錯誤新增
        IDObj["Yahoo"]["web"]= ""
    try:#typeid
        IDObj["Yahoo"]["type"]=prdidData[getArow][5]
    except:#錯誤新增
        IDObj["Yahoo"]["type"]=""
    try:#webid
        IDObj["Ruten"]["web"]=prdidData[getArow][6]
    except:#錯誤新增
        IDObj["Ruten"]["web"]= ""
    try:#typeid
        IDObj["Ruten"]["type"]=prdidData[getArow][7]
    except:#錯誤新增
        IDObj["Ruten"]["type"]=""    
    try:#webid
        IDObj["Shopee"]["web"]=prdidData[getArow][8]
    except:#錯誤新增
        IDObj["Shopee"]["web"]= ""
    try:#typeid
        IDObj["Shopee"]["type"]=prdidData[getArow][9]
    except:#錯誤新增
        IDObj["Shopee"]["type"]=""    
    try:#webid
        IDObj["PCt"]["web"]=prdidData[getArow][10]
    except:#錯誤新增
        IDObj["PCt"]["web"]= ""
    try:#typeid
        IDObj["PCt"]["type"]=prdidData[getArow][11]
    except:#錯誤新增
        IDObj["PCt"]["type"]=""
    try:#webid
        IDObj["PCd"]["web"]=prdidData[getArow][12]
    except:#錯誤新增
        IDObj["PCd"]["web"]= ""
    try:#typeid
        IDObj["PCd"]["type"]=prdidData[getArow][13]
    except:#錯誤新增
        IDObj["PCd"]["type"]=""
    try:#webid
        IDObj["pcone"]["web"]=prdidData[getArow][14]
    except:#錯誤新增
        IDObj["pcone"]["web"]= ""
    try:#typeid
        IDObj["pcone"]["type"]=prdidData[getArow][15]
    except:#錯誤新增
        IDObj["pcone"]["type"]=""
        
        
        
    #找yahoo
    try:
        if IDObj["Yahoo"]["web"] : #有web id開始找
            gotstock = getyahoo(IDObj["Yahoo"]["web"])
            if type(gotstock["prd"]) == type("str"): #單一商品
                wval[0] = gotstock["prd"] #寫入數量
            else: #多商品
                for i in range(len(gotstock["prd"])):
                    if gotstock["prd"][i][0] == IDObj["Yahoo"]["type"]:
                         wval[0] = gotstock["prd"][i][1]
                if wval[0] == "":
                    wval[0] = "款式ID錯誤"
        else:
            wval[0] = "noID"
        if gotstock["state"] == "3":
             wval[0] ="下架"
    except:
        wval[0] ="錯誤"
    #找Ruten
    try:
        if IDObj["Ruten"]["web"] : #有web id開始找
            gotstock = getruten(IDObj["Ruten"]["web"])
            if type(gotstock["prd"]) == type("str"): #單一商品
                wval[1] = gotstock["prd"] #寫入數量
            else: #多商品
                for i in range(len(gotstock["prd"])):
                    if gotstock["prd"][i][0] == IDObj["Ruten"]["type"]:
                        wval[1] = gotstock["prd"][i][1]
                        if gotstock["prd"][i][3] == "N": #如果款式關閉
                            wval[1] = "關閉"
                if wval[1] == "":
                    wval[1] = "款式ID錯誤"
        else:
            wval[1] = "noID"
        if gotstock["state"] == False:
             wval[1] ="下架"
    except:
        wval[1] ="錯誤"
        
    #找蝦皮
    try:
        if IDObj["Shopee"]["web"] : #有web id開始找
            gotstock = getshopee(IDObj["Shopee"]["web"])
            if type(gotstock["prd"]) == type("str"): #單一商品
                wval[5] = gotstock["prd"] #寫入數量
            else: #多商品
                for i in range(len(gotstock["prd"])):
                    if gotstock["prd"][i][0] == IDObj["Shopee"]["type"]:
                         wval[5] = gotstock["prd"][i][1]
                if wval[5] == "":
                    wval[5] = "款式ID錯誤"
        else:
            wval[5] = "noID"
        if gotstock["state"] == "8":
             wval[5] ="下架"
    except:
        del wval[5]
        print("蝦皮錯誤")
        
    
    #找PC梓原
    try:
        if IDObj["PCt"]["web"] : #有web id開始找
            gotstock = getpct(IDObj["PCt"]["web"]) #查找網頁
            for i in range(len(gotstock["prd"])):
                if gotstock["prd"][i][0] == IDObj["PCt"]["type"]:
                    wval[3] = gotstock["prd"][i][1]
            if wval[3] == "":
                wval[3] = "款式ID錯誤"

        else:
            wval[3] = "noID"

    except:
        wval[3] ="錯誤或下架"

    #找PC大一
    try:
        if IDObj["PCd"]["web"] : #有web id開始找
            gotstock = getpcd(IDObj["PCd"]["web"]) #查找網頁
            for i in range(len(gotstock["prd"])):
                if gotstock["prd"][i][0] == IDObj["PCd"]["type"]:
                    wval[4] = gotstock["prd"][i][1]
            if wval[4] == "":
                wval[4] = "款式ID錯誤"

        else:
            wval[4] = "noID"

    except:
        wval[4] ="錯誤或下架"
        
    #找松果
    try:
        if IDObj["pcone"]["web"] : #有web id開始找
            gotstock = getpcone(IDObj["pcone"]["web"]) #查找網頁
            for i in range(len(gotstock["prd"])):
                if gotstock["prd"][i][0] == IDObj["pcone"]["type"]:
                    wval[2] = gotstock["prd"][i][1]
            if wval[2] == "":
                wval[2] = "款式ID錯誤"

        else:
            wval[2] = "noID"

    except:
        wval[2] ="錯誤或下架"
    
    
    writesheet("庫存表!G"+str(getArow+1),[wval])
    return  [wval]    
 
def getyahoo(pid):
    url = "https://tw.bid.yahoo.com/item/"+pid
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    #for item in soup.select(".listing-title"): #html 可以使用select 選擇想要的東西
    #print(item.select("a")[0].text)
    gotjson = soup.select("#isoredux-data")[0].get("data-state") #get 取得屬性
    load = json.loads(gotjson)  #轉成python dict
    outObj = {"state":"0","prd":""}
    if len(load["item"]["specs"]) == 0:
        #print(load["item"]["models"][0]["id"]+":"+ str(load["item"]["models"][0]["qty"]))
        outObj["prd"] = str(load["item"]["models"][0]["qty"])
    else:
        outAry = []
        for i in range(len(load["item"]["models"])): #提取ID
            specnameID = str(load["item"]["models"][i]["specCombination"]).split(":")
            specname = str(load["item"]["specs"][0]['options'][i]['name'])
            outAry.append([str(load["item"]["models"][i]["id"]),str(load["item"]["models"][i]["qty"]),specname])
            #print(load["item"]["models"][i]["id"]+":"+ str(load["item"]["models"][i]["qty"])+" "+specname)
            outObj["prd"] = outAry
            
    outObj["state"] = str(load["item"]["status"])    
    return outObj
    #print(str(load["item"]["status"])+"   ● 2:上架,3:下架")

def getshopee(pid):
    global browser
    load = "" #重製
    browser.get('https://shopee.tw/product/2019696/'+pid) 
    time.sleep(2) #第一次讀取會失敗 可能要加點選事件，或是觀看視窗的事件 資料可能放在dataLayer[5] #判斷4有無值，無值則layer[5]
    browser.find_element_by_css_selector("input.shopee-button-outline").click()
    browser.execute_script("return document.getElementsByClassName('shopee-product-info__header__title')[0].innerHTML = JSON.stringify(dataLayer[4].info.impressions[0].targetData.item)")
    outObj = {"state":"0","prd":""}
    prdAry = []
    if browser.find_element_by_class_name('shopee-product-info__header__title').text == 'undefined': #如果沒有值，則找layer5
        print('ly5')
        browser.execute_script("return document.getElementsByClassName('shopee-product-info__header__title')[0].innerHTML = JSON.stringify(dataLayer[5].info.impressions[0].targetData.item)")
    gotjson = browser.find_element_by_class_name('shopee-product-info__header__title').text #取得JSON
    load = json.loads(gotjson)
    if len(load['models']) == 0: #判斷有沒有款式
        outObj["prd"] = str(load['stock'])
    else:
        for i in range(len(load['models'])):
            prdAry.append([str(load['models'][i]['modelid']),str(load['models'][i]['stock']),str(load['models'][i]['name'])])
        outObj["prd"] = prdAry
    outObj["state"] = str(load['status'])
    return outObj
    
            
def getruten(pid):
    url = "https://goods.ruten.com.tw/item/show?"+pid
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    strnum = soup.select('script[type="text/javascript"]')[16].text.find("RT.context = ") #提取JSON資料 #find 找到RT.context的位址
    gotjson = soup.select('script[type="text/javascript"]')[16].text[strnum+len("RT.context = "):-2]
    load = json.loads(gotjson)  #轉成python dict
    outObj = {"state":"0","prd":""}
    prdAry = []
    if 'spec_info' in load.keys(): #是否有款式
        for k,v in load['spec_info']['specs'].items():#for 出dict 資料
            prdAry.append([k,v['spec_num'],v['spec_name'],v["spec_status"]])
            #print(k+':'+v['spec_num']+" "+v['spec_name']+v["spec_status"])
        outObj["prd"]=prdAry
    else:
        outObj["prd"]=str(load['remain_count'])
        #print(load['remain_count'])
    outObj["state"] = load['is_product_buyer']
    return outObj

def getpct(pid):
    url = "http://seller.pcstore.com.tw/S163498400/"+pid+".htm"
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    gotjson = soup.select('#specs')[0].text
    load = json.loads(gotjson)
    outObj = {"state":"0","prd":""}
    prdAry = []
    if type(load) == dict: #新上的都是字典檔
        for k,v in load.items():
            prdAry.append([v['p_sseq'],v['p_invt'],['p_spec']])
    else:#舊的都是陣列
        for i in range(len(load)):
            prdAry.append([load[i]['p_sseq'],load[i]['p_invt'],load[i]['p_spec']])
    outObj["prd"] = prdAry
    return outObj


def getpcd(pid):
    url = "http://seller.pcstore.com.tw/S188431702/"+pid+".htm"
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    gotjson = soup.select('#specs')[0].text
    load = json.loads(gotjson)
    outObj = {"state":"0","prd":""}
    prdAry = []
    if type(load) == dict: #新上的都是字典檔
        for k,v in load.items():
            prdAry.append([v['p_sseq'],v['p_invt'],['p_spec']])
    else:#舊的都是陣列
        for i in range(len(load)):
            prdAry.append([load[i]['p_sseq'],load[i]['p_invt'],load[i]['p_spec']])
    outObj["prd"] = prdAry
    return outObj
    
def getpcone(pid):
    url = "https://www.pcone.com.tw/product/info/"+pid
    my_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    gotrq = requests.get(url,headers = my_headers)
    soup = BeautifulSoup(gotrq.text, "html.parser") #"html.parser" html解析器 將html 轉為bs4格式操作
    strnum = soup.select('script')[23].text.find("window._pc_p = ") #提取JSON資料
    gotjson = soup.select('script')[23].text[strnum+len("window._pc_p = "):-2]
    load = json.loads(gotjson)  #轉成python dict
    outObj = {"state":"0","prd":""}
    prdAry = []
    for i in range(len(load['volumes'])):
        prdAry.append([str(load['volumes'][i]['volume_id']),str(load['volumes'][i]['volume_remaining']),str(load['volumes'][i]['option'])])
    outObj["prd"] = prdAry
    return outObj

def getsheet(sheetrange):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
        creds = tools.run_flow(flow, store)
    service = build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    SPREADSHEET_ID = '19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI'
    RANGE_NAME = sheetrange
    result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID,
                                                range=RANGE_NAME).execute()
    values = result.get('values', [])
    if not values:
        print('No data found.')
    else:
        return values

def writesheet(sheetrange,writeVal):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
        creds = tools.run_flow(flow, store)
    service = build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    SPREADSHEET_ID = '19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI'
    RANGE_NAME = sheetrange
    values =[['=52+25']]
    body = {'values': writeVal}
    value_input_option = 'USER_ENTERED'
    result = service.spreadsheets().values().update(spreadsheetId=SPREADSHEET_ID,range=RANGE_NAME,valueInputOption=value_input_option,body=body).execute()
    print(result)

def delsheet(sheetrange):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
        creds = tools.run_flow(flow, store)
    service = build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    SPREADSHEET_ID = '19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI'
    RANGE_NAME = {'ranges' : [sheetrange]}
    result = service.spreadsheets().values().batchClear(spreadsheetId=SPREADSHEET_ID,body=RANGE_NAME).execute()
    print(result)

In [ ]:
import xlrd
getidlist = getsheet("商品ID!I:J")
oklist = shopeeExcel("a.xlsx") #讀取EXCEL

#提取sheet 資料
sprdid = []
smodelid = []
for i in range(len(getidlist)):
    try:
        sprdid.append(getidlist[i][0])
    except:
        sprdid.append("")
    try:
        smodelid.append(getidlist[i][1])
    except:
        smodelid.append("")
        
#尋找並寫入sheet
wrval = []
noidlist = []

for i in range(len(getidlist)):
    wrval.append([""])
for i in range(len(oklist)):

    try:
        findrow = sprdid.index(oklist[i]["ID"])
        wrval[findrow] = [oklist[i]["stock"]]
    except:
        try:
            findrow = smodelid.index(oklist[i]["ID"])
            wrval[findrow] = [oklist[i]["stock"]]
        except:
            noidlist.append(oklist[i]["ID"])
            
del wrval[0]
writesheet("庫存表!L2",wrval)


In [1]:
def shopeeExcel(path):
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(path)
    table = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table.nrows #取总行数
    prdid = []
    for row in range(rows_count):#無款式
        if table.cell(row,0).value:
            try:
                prdid.append({"ID":str(int(table.cell(row,0).value)),"stock":str(int(table.cell(row,6).value))})
            except:
                a = "a"
    for i in range(20):
        col = 5*i
        for row in range(rows_count): #商品款式
            try:
                prdid.append({"ID":str(int(table.cell(row,8+col).value)),"stock":str(int(table.cell(row,12+col).value))})
            except:
                a = "b"
    return prdid